# Plots

In [1]:
# Import to be able to import python package from src
import sys
sys.path.insert(0, '../src')

In [2]:
import pandas as pd
import ontime as on
from darts.datasets import EnergyDataset

The `LightGBM` module could not be imported. To enable LightGBM support in Darts, follow the detailed instructions in the installation guide: https://github.com/unit8co/darts/blob/master/INSTALL.md


The `Prophet` module could not be imported. To enable Prophet support in Darts, follow the detailed instructions in the installation guide: https://github.com/unit8co/darts/blob/master/INSTALL.md


---
## Load data

In [3]:
ts = EnergyDataset().load()

### Complete TimeSeries

In [4]:
df = ts.pd_dataframe()
df = df.interpolate()
cols = ['generation biomass', 'generation solar', 'generation nuclear']
df = df[cols]

In [5]:
ts = on.TimeSeries.from_dataframe(df)

### Prepare data

In [6]:
ts_uni = ts['generation solar'].slice(pd.Timestamp('2015'), pd.Timestamp('2016'))
ts_multi = ts.slice(pd.Timestamp('2015'), pd.Timestamp('2016'))

---
## Plots

https://medium.com/nightingale/style-guidelines-92ebe166addc

In [7]:
import altair as alt

# To show big time series
alt.data_transformers.enable("vegafusion")

DataTransformerRegistry.enable('vegafusion')

### Define theme

In [8]:
alt.themes.names

<bound method PluginRegistry.names of ThemeRegistry(active='default', registered=['dark', 'default', 'excel', 'fivethirtyeight', 'ggplot2', 'googlecharts', 'latimes', 'none', 'opaque', 'powerbi', 'quartz', 'urbaninstitute', 'vox'])>

In [9]:
alt.themes.enable('default')

ThemeRegistry.enable('default')

### TimeSeries Plot

In [10]:
def plot(ts):
    
    # Transform data
    df = ts.pd_dataframe()
    df = df.reset_index()
    df = df.melt('time', var_name='variable', value_name='value')

    # Plot
    chart = alt.Chart(df).mark_line().encode(
        x='time:T',
        y='value:Q',
        color='variable:N',
    ).properties(
        width=500,
        height=300
    )

    return chart

#### Univariate TimeSeries

In [11]:
plot(ts_uni)

alt.Chart(...)

#### Multivariate TimeSeries

In [12]:
plot(ts_multi)

alt.Chart(...)

### Heatmap Plot

In [13]:
def heatmap(ts):
    # Transform data
    df = ts.pd_dataframe()
    df = df.reset_index()
    df = df.melt('time', var_name='variable', value_name='value')
    
    color_condition = alt.condition(
        "month(datum.value) == 1 && date(datum.value) == 1",
        alt.value("black"),
        alt.value(None),
    )
    
    chart = alt.Chart(df).mark_rect().encode(
        alt.X("yearmonthdate(time):O")
            .title("Time")
            .axis(
                format="%Y",
                labelAngle=0,
                labelOverlap=False,
                labelColor=color_condition,
                tickColor=color_condition,
            ),
        alt.Y("variable:N").title(None),
        alt.Color("sum(value)").title("Value")
    ).properties(
        width=500,
        height=100
    )

    return chart

#### Univariate Heatmap

In [14]:
heatmap(ts_uni)

alt.Chart(...)

#### Multivariate Heatmap

In [15]:
heatmap(ts_multi)

alt.Chart(...)

### Prediction Plot

Quickly make a prediction

In [16]:
train, test = ts_uni.split_after(pd.Timestamp('2015-09-01'))

In [17]:
gp = on.context.common.GenericPredictor()
gp.fit(train);

In [18]:
pred = gp.predict(48)

#### Univariate prediction plot

In [19]:
def plot_prediction(train_ts, pred_ts=None, test_ts=None):
    
    # Train section
    df_train = train_ts.pd_dataframe()
    df_train = df_train.reset_index()
    df_train = df_train.melt('time', var_name='variable', value_name='value')
    df_train['variable'] = 'Training set'

    # Prediction section
    df_pred = pred_ts.pd_dataframe()
    df_pred = df_pred.reset_index()
    df_pred = df_pred.melt('time', var_name='variable', value_name='value')
    df_pred['variable'] = 'Prediction'

    # Test section
    df_test = test_ts.pd_dataframe()
    df_test = df_test.reset_index()
    df_test = df_test.melt('time', var_name='variable', value_name='value')
    df_test['variable'] = 'Truth'

    df = pd.concat([df_train, df_pred, df_test])
    
    chart = alt.Chart(df).mark_line().encode(
        x='time:T',
        y='value:Q',
        color='variable:N',
        opacity=alt.condition(
            alt.datum.variable == 'Truth', 
            alt.value(0.5),
            alt.value(1.0)
        ),
        strokeDash=alt.condition(
            alt.datum.variable == 'Truth',
            alt.value([5, 2]),  # Dash pattern: 5 units of line followed by 2 units of gap
            alt.value([0])      # Solid line
        )
    ).properties(
        width=500,
        height=300
    )

    return chart

In [20]:
plot_prediction(train[-96:], pred, test[:48])

alt.Chart(...)

## Anomaly Plot

Quickly detect anomlies

In [21]:
gd = on.context.common.GenericDetector()
gd.fit(train);

In [22]:
detected_train = gd.detect(train)
detected_test = gd.detect(test)
predetected = gd.predetect(10)

In [23]:
# params
train_ts = train[-96:]
test_ts = test[:48]
detected_train_ts = detected_train[-96:]
detected_test_ts = detected_test[:48]
predetected_ts = predetected[:48]

In [24]:
df_train = train_ts.pd_dataframe()
df_train = df_train.reset_index()
df_train = df_train.melt('time', var_name='variable', value_name='value')
df_train['variable'] = 'signal_train'

In [25]:
df_test = test_ts.pd_dataframe()
df_test = df_test.reset_index()
df_test = df_test.melt('time', var_name='variable', value_name='value')
df_test['variable'] = 'signal_test'

In [26]:
df_detected_train = detected_train_ts.pd_dataframe()
df_detected_train = df_detected_train.reset_index()
df_detected_train = df_detected_train.melt('time', var_name='variable', value_name='value')
df_detected_train['variable'] = 'anomaly_train'

In [27]:
df_detected_test = detected_test_ts.pd_dataframe()
df_detected_test = df_detected_test.reset_index()
df_detected_test = df_detected_test.melt('time', var_name='variable', value_name='value')
df_detected_test['variable'] = 'anomaly_test'

In [28]:
df_predetected = predetected_ts.pd_dataframe()
df_predetected = df_predetected.reset_index()
df_predetected = df_predetected.melt('time', var_name='variable', value_name='value')
df_predetected['variable'] = 'anomaly_predetected'

In [29]:
df = pd.concat([df_train, df_test, df_detected_train, df_detected_test, df_predetected])

In [30]:
# Pivot data to wide form
df_wide = df.pivot(index='time', columns='variable', values='value').reset_index()

In [31]:
chart_signal_train = alt.Chart(df_wide).mark_line().encode(
    x='time:T',
    y='signal_train:Q'
).properties(
    width=500,
    height=300
)

In [32]:
chart_signal_test = alt.Chart(df_wide).mark_line(color='lightblue').encode(
    x='time:T',
    y='signal_test:Q'
)

In [33]:
chart_anomaly_train = alt.Chart(df_wide).transform_filter(
    alt.datum.anomaly_train == 1
).mark_circle(color='red', size=60).encode(
    x='time:T',
    y='signal_train:Q',
)

In [34]:
chart_anomaly_test = alt.Chart(df_wide).transform_filter(
    alt.datum.anomaly_test == 1
).mark_circle(color='red', size=60).encode(
    x='time:T',
    y='signal_test:Q',
)

In [35]:
chart_anomaly_predetected = alt.Chart(df_wide).transform_filter(
    alt.datum.anomaly_predetected == 1
).mark_circle(color='green', size=60).encode(
    x='time:T',
    y='signal_test:Q',
)

In [36]:
chart_signal_train + chart_signal_test + chart_anomaly_train + chart_anomaly_test + chart_anomaly_predetected

alt.LayerChart(...)

In [37]:
def plot_anomalies(ts, ts_anomaly):

    df = ts.pd_dataframe()
    df = df.reset_index()
    df = df.melt('time', var_name='variable', value_name='value')
    df['variable'] = 'signal'

    df_anomaly = ts_anomaly.pd_dataframe()
    df_anomaly = df_anomaly.reset_index()
    df_anomaly = df_anomaly.melt('time', var_name='variable', value_name='value')
    df_anomaly['variable'] = 'anomaly'

    df = pd.concat([df, df_anomaly])
    df = df.pivot(index='time', columns='variable', values='value').reset_index()

    chart_signal = alt.Chart(df).mark_line().encode(
        x='time:T',
        y='signal:Q'
    ).properties(
        width=500,
        height=300
    )
    
    chart_anomaly = alt.Chart(df).transform_filter(
        alt.datum.anomaly == 1
    ).mark_circle(color='red', size=60).encode(
        x='time:T',
        y='signal:Q',
    )

    return chart_signal + chart_anomaly


In [38]:
plot_anomalies(test_ts, detected_test_ts) + plot_anomalies(test_ts, predetected_ts)

alt.LayerChart(...)

## Study Seaborn new API

Example of the syntax


    (
        so.Plot(penguins, x="species", y="body_mass_g", color="sex")
        .add(so.Dot(pointsize=10), so.Agg())
    )

With this API, you always add a plot with `so.Plot(data, x=str, y=str)`, then you add something to it with the `.add(Mark, Stat)` function.

`Mark` is an object that graphically represents data values. For instance : `so.Bar()`, `so.Line()`, `so.Dot()`.

`Stat` is an object that is a statistical transformation. For instance : `so.Agg()`, `so.Dodge()`, `so.Jitter(.3)`, `so.Hist()`.

In the add function, only one Mark can be added but as many Stat as desired can be added as parameter.|

